# Capítulo 07 — Instruction Tuning: Criando um Assistente

Neste capítulo, atingimos o ápice da nossa jornada. Vamos transformar um modelo que apenas "completa frases" em um **assistente que segue instruções**.

--- 
### 🧪 O conceito de Alinhamento
Modelos base aprendem a probabilidade estatística das palavras. Mas eles não sabem que, quando um humano faz uma pergunta, ele espera uma resposta direta. O **Supervised Fine-Tuning (SFT)** é o processo de mostrar ao modelo milhares de exemplos de `[Comando]` -> `[Resposta]`. 

![Pipeline SFT](./infograficos/04-pipeline-sft.png)

## 1. Setup e Persistência de Dados

**⚠️ Importante:** Como o Google Colab apaga os arquivos ao fechar a sessão, se você não tiver o `gpt_checkpoint.pt` do Capítulo 05 aqui, o modelo começará do zero (sem saber falar português).

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from lib.gptmini import GPTConfig, GPTMini

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Executando em: {device}")

# Se o arquivo não existir, vamos criar um aviso claro
if not os.path.exists("gpt_checkpoint.pt"):
    print("❌ CHECKPOINT NÃO ENCONTRADO!")
    print("Para melhores resultados, faça o upload do arquivo 'gpt_checkpoint.pt' gerado no Capítulo 05.")
else:
    print("✨ Checkpoint detectado e pronto para carregar.")

## 2. Dataset de Instruções Curado

Para que o modelo responda algo com sentido, precisamos de um dataset que cubra os tópicos que vamos testar. Vamos ensinar o modelo a ser um mini-especialista em IA.

In [ ]:
dataset = [
    {
        "instruction": "Explique Machine Learning",
        "response": "Machine Learning e o estudo de algoritmos que aprendem padroes de dados."
    },
    {
        "instruction": "O que e um LLM?",
        "response": "LLM e um modelo de linguagem treinado em grandes volumes de texto para entender e gerar linguagem."
    },
    {
        "instruction": "Defina Inteligencia Artificial",
        "response": "IA e a simulacao de processos de inteligencia humana por maquinas e sistemas de computacao."
    },
    {
        "instruction": "O que e o mecanismo de Atencao?",
        "response": "Atencao permite que o modelo foque em partes importantes do texto para entender o contexto."
    }
]

def format_instruction(item):
    return f"### Comando:\n{item['instruction']}\n\n### Resposta:\n{item['response']}"

all_text = "".join([format_instruction(d) for d in dataset])
chars = sorted(set(all_text))
stoi = {c:i for i,c in enumerate(chars)}
itos = {i:c for c,i in stoi.items()}
vocab_size = len(chars)

def encode(text): return [stoi[c] for c in text if c in stoi]
def decode(tokens): return "".join([itos[t] for t in tokens])

print(f"Tamanho do Vocabulario: {vocab_size} caracteres")

## 3. Inicialização com Transferência de Pesos

Vamos carregar a "inteligência base" do Capítulo 05. Mesmo que o vocabulário seja um pouco diferente, os **Transformer Blocks** (as camadas de atenção) carregam o conhecimento estrutural da língua.

In [ ]:
config = GPTConfig(vocab_size=vocab_size, context_size=64, d_model=128, n_heads=4, n_layers=2)
backbone = GPTMini(config).to(device)

try:
    ckpt = torch.load("gpt_checkpoint.pt", map_location=device)
    state_dict = ckpt["state_dict"] if "state_dict" in ckpt else ckpt
    model_dict = backbone.state_dict()
    # Carrega apenas camadas com shapes idênticos
    pretrained_dict = {k: v for k, v in state_dict.items() if k in model_dict and v.shape == model_dict[k].shape}
    backbone.load_state_dict(pretrained_dict, strict=False)
    print(f"🧠 Inteligência Base carregada: {len(pretrained_dict)} camadas reaproveitadas.")
except:
    print("🌱 Iniciando treinamento do zero (sem pesos prévios).")

## 4. O Coração do Capítulo: Mascaramento da Loss

No Instruction Tuning, não queremos que o modelo aprenda a prever a pergunta (nós já sabemos a pergunta!). Queremos que ele foque 100% na resposta. 

Usamos uma **Máscara Binária** onde a Loss do Comando é multiplicada por 0 e a da Resposta por 1.

![Masking Loss](./infograficos/03-mascaramento-loss-resposta.png)

In [ ]:
def build_dataset(data, context_size=64):
    X, Y, masks = [], [], []
    for item in data:
        cmd = f"### Comando:\n{item['instruction']}\n\n### Resposta:\n"
        resp = item["response"]
        full = cmd + resp
        
        full_tokens = encode(full)
        cmd_tokens = encode(cmd)
        
        for i in range(len(full_tokens) - context_size):
            x = full_tokens[i : i+context_size]
            y = full_tokens[i+1 : i+context_size+1]
            # Máscara: 0 para o que for comando, 1 para o que for resposta
            m = [0]*len(cmd_tokens) + [1]*(context_size - len(cmd_tokens))
            
            X.append(x); Y.append(y); masks.append(m[:context_size])
            
    return torch.tensor(X).to(device), torch.tensor(Y).to(device), torch.tensor(masks).to(device)

X, Y, MASK = build_dataset(dataset)

class InstructWrapper(nn.Module):
    def __init__(self, gpt): 
        super().__init__(); self.gpt = gpt
    def forward(self, x, y=None, m=None):
        logits, _ = self.gpt(x)
        loss = None
        if y is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), y.view(-1), reduction='none')
            loss = (loss * m.view(-1)).mean()
        return logits, loss

model = InstructWrapper(backbone).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)

## 5. Treinamento e Monitoramento

Vamos ver o modelo "se alinhando" às nossas instruções.

In [ ]:
loss_history = []
print("🚀 Alinhando o modelo...")

for step in range(401):
    idx = torch.randint(0, X.size(0), (8,))
    logits, loss = model(X[idx], Y[idx], MASK[idx])
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    loss_history.append(loss.item())
    if step % 100 == 0: print(f"Step {step:03d} | Loss: {loss.item():.4f}")

plt.figure(figsize=(8, 3))
plt.plot(loss_history, color='#34A853')
plt.title("Curva de Alinhamento (SFT Loss)")
plt.show()

## 6. O Teste Final: O Assistente em Ação

Agora, o modelo deve ser capaz de reconhecer o formato de comando e gerar uma resposta baseada no que aprendeu.

In [ ]:
@torch.no_grad()
def generate_response(model, instruction, max_new=50):
    model.eval()
    context = f"### Comando:\n{instruction}\n\n### Resposta:\n"
    tokens = torch.tensor(encode(context)).unsqueeze(0).to(device)
    
    for _ in range(max_new):
        idx_cond = tokens[:, -64:]
        logits, _ = model(idx_cond)
        next_token = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
        tokens = torch.cat([tokens, next_token], dim=1)
        if itos[next_token.item()] == "\n": break
        
    return decode(tokens[0].tolist())

print("🤖 TESTE DO ASSISTENTE:")
print("-" * 30)
pergunta = "Explique Machine Learning"
print(generate_response(model, pergunta))

## 🏁 Conclusão da Jornada

Parabéns! Você construiu um LLM do zero absoluto até o alinhamento de instruções. 

O que vimos hoje é a base de como empresas como a OpenAI e o Google treinam seus assistentes. A diferença é apenas a escala (bilhões de parâmetros e bilhões de exemplos), mas a matemática e a lógica do **Mascaramento de Loss** e do **SFT** são exatamente estas.

![Avaliação](./infograficos/05-avaliacao-respostas.png)